In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import binary_erosion
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

In [3]:
def getBorderColor(img):
    
    border = np.asarray(img[0,:])
    border = np.concatenate((border, np.asarray(img[-1, :])))
    border = np.concatenate((border, np.asarray(img[:, 0])))
    border = np.concatenate((border, np.asarray(img[:, -1])))
    return np.bincount(border).argmax()

def pre_processing(img):
    #img_blurred = cv2.medianBlur(img,5)
    #img_binarized = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    _, img_binarized = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)    
    if(getBorderColor(img_binarized) != 0):
        img_binarized = cv2.bitwise_not(img_binarized) 
    #img_binarized = adaptiveThresh(img, t=15, div=8)
    return img_binarized

In [4]:

def showImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    #print(getBorderColor(img))
    #w, h = img.shape[:2]
    #print(cv2.countNonZero(img)/(w*h) * 100)


#arr = np.array([[1,2,3], [4,5,6], [7,8,9]])
#getBorderColor(arr)

In [4]:
def getFeaturesHOG(img):
   img = cv2.resize(img, (256, 128))
   cell_size = (32, 32)  # h x w in pixels
   block_size = (2, 2)  # h x w in cells
   nbins = 9  # number of orientation bins
   # winSize is the size of the image cropped to an multiple of the cell size
   hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                    img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

   hog_feats = hog.compute(img)
   return hog_feats.flatten() 

In [5]:
# feature f1 (density of foreground (black) pixels.)

img_height = X_train[0].shape[0]
img_width = X_train[0].shape[1]
f1 = []
# Crop out the window and calculate f1 for each cell
for r in range(img_width - 1, 8 - 1, -4):
    foreground = []
    for c in range(0, img_height, int(img_height / 20)):
        window = X_train[0][c:c+int(img_height / 20)-1, r-8:r]
        foreground.append(np.average(window == 255))
    f1.append(np.sum(foreground))
print(f1)


NameError: name 'X_train' is not defined

In [ ]:
# number of transitions black/white between two consecutive cells.
f2 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    cells = []
    #get the cells in each window
    for c in range(0, img_height, HEIGHT):
        cells.append(image[c:c+HEIGHT-1, r-WIDTH:r])
    summtion = 0
    for i in range(2,N):
        bi = int(np.sum(cells[i] == 255) == 0)
        b_i = int(np.sum(cells[i - 1] == 255) == 0)
        summtion += np.abs(bi - b_i )
    f2.append(summtion)
        


In [ ]:
def get_center_of_mass(window):
    # calculate moments of binary image
    M = cv2.moments(window)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / (M["m00"] + 1e-5))
    cY = int(M["m01"] / (M["m00"] + 1e-5))
    return cX,cY


In [ ]:
# f3 difference’s position of gravity centers of foreground pixels in two consecutive frame (current and previous) note c - c_prev
f3 = []
centers = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    centers.append(get_center_of_mass(image[:, r-WIDTH:r]))
f3.append(np.sqrt(np.power(centers[0][0], 2) + np.power(centers[0][1], 2)))
for i in range(1,len(centers)):
    f3.append(np.sqrt(np.power(centers[i][0] - centers[i - 1][0],
                       2) + np.power(centers[i][1] - centers[i - 1][1], 2)))

In [ ]:
# f4 normalized vertical position of the center of gravity of the foreground pixels in the whole frame with respect tothe lower baseline. c - c_prev
f4 = []
centers = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    centers.append(get_center_of_mass(image[:, r-WIDTH:r]))
for i in range(0, len(centers)):
    f4.append(np.power(centers[i][1] - lb, 2) / img_height)


In [ ]:
#f5,f6 represent the density of foreground pixels over and under the lower baselines.
f5 = []
f6 = []
print(lb)
for r in range(img_width - 1, WIDTH - 1, -STRID):
    f5.append(np.sum(image[0:lb-1, r-WIDTH:r] == 255) / (img_height * WIDTH))
    f6.append(np.sum(image[lb:, r-WIDTH:r] == 255) / (img_height * WIDTH))


In [ ]:
k = -1
index = 0
for c in range(0, img_height, HEIGHT):
    if c >= lb:
        k = c
        break
    index += 1

f7 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    cells = []
    #get the cells in each window
    for c in range(0, k, HEIGHT):
        cells.append(image[c:c+HEIGHT-1, r-WIDTH:r])
    summtion = 0
    for i in range(2, len(cells)):
        bi = int(np.sum(cells[i] == 255) == 0)
        b_i = int(np.sum(cells[i - 1] == 255) == 0)
        summtion += np.abs(bi - b_i)
    f7.append(summtion)

In [ ]:
# zone to which the gravity center of black pixels belongs (lower zone f8 = 3, middle zone f8 = 2, upper zone f8 = 1)
f8 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    center_y = get_center_of_mass(image[:, r-WIDTH:r])[1]
    if center_y > lb:
        f8.append(3)
    elif center_y < lb and center_y > lu:
        f8.append(2)
    else:
        f8.append(1)

In [ ]:
# Cleft-up: the number of background pixels that have neighbor black pixels in the two directions (left and up)
#The same applies to f9, . . ., f14 in six directions left-up, up-right, right-down, down-left, vertical and horizontal.f15, . . ., f20
f9 = []
f10 = []
f11 = []
f12 = []
f13 = []
f14 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    window = image[:, r-WIDTH:r]
    k1 = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]
    ])
    k2 = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 1, 0]
    ])
    
    f9.append(np.sum(binary_erosion(255 - window, k2) * binary_erosion(window, k1))/ img_height)
    
    k1 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0]
    ])
    k2 = np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1]
    ])    
    f10.append(np.sum(binary_erosion(255 - window, k2) * binary_erosion(window, k1))/ img_height)
    
    
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])
    k2 = np.array([
        [0, 1, 1],
        [1, 1, 0],
        [1, 0, 0]
    ])    
    f11.append(np.sum(binary_erosion(255 - window, k2) * binary_erosion(window, k1))/ img_height)
    
    k1 = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]
    ])
    k2 = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]
    ])    
    f12.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / img_height)
    
    k1 = np.array([
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]
    ])
    k2 = np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ])    
    f13.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / img_height)
    
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 1]
    ])
    k2 = np.array([
        [0, 0, 0],
        [1, 1, 1],
        [0, 0, 0]
    ])    
    f14.append(np.sum(binary_erosion(255 - window, k2) * binary_erosion(window, k1)) / img_height)


In [ ]:
# Cleft-up: the number of background pixels that have neighbor black pixels in the two directions (left and up)
#The same applies to f9, . . ., f14 in six directions left-up, up-right, right-down, down-left, vertical and horizontal.f15, . . ., f20
f15 = []
f16 = []
f17 = []
f18 = []
f19 = []
f20 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    window = image[lu:lb, r-WIDTH:r]
    k1 = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]
    ])
    k2 = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 1, 0]
    ])
    
    f15.append(np.sum(binary_erosion(255 - window, k2) * binary_erosion(window, k1))/ np.abs(lb-lu))
    
    k1 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0]
    ])
    k2 = np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1]
    ])    
    f16.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / np.abs(lb-lu))
    
    
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])
    k2 = np.array([
        [0, 1, 1],
        [1, 1, 0],
        [1, 0, 0]
    ])    
    f17.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / np.abs(lb-lu))
    
    k1 = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]
    ])
    k2 = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]
    ])    
    f18.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / np.abs(lb-lu))
    
    k1 = np.array([
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]
    ])
    k2 = np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ])    
    f19.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / np.abs(lb-lu))
    
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 1]
    ])
    k2 = np.array([
        [0, 0, 0],
        [1, 1, 1],
        [0, 0, 0]
    ])    
    f20.append(np.sum(binary_erosion(255 - window, k2) *
                      binary_erosion(window, k1)) / np.abs(lb-lu))


In [ ]:
f21 = []
f22 = []
f23 = []
f24 = []
f25 = []
f26 = []
f27 = []
f28 = []

for r in range(img_width - 1, WIDTH - 1, -STRID):
    window = image[lu:lb, r-WIDTH:r]
    f21.append(np.sum(window[:, 0]) / img_height)
    f22.append(np.sum(window[:, 1]) / img_height)
    f23.append(np.sum(window[:, 2]) / img_height)
    f24.append(np.sum(window[:, 3]) / img_height)
    f25.append(np.sum(window[:, 4]) / img_height)
    f26.append(np.sum(window[:, 5]) / img_height)
    f27.append(np.sum(window[:, 6]) / img_height)
    f28.append(np.sum(window[:, 7]) / img_height)




In [64]:
# pre processing of the image (baseline estimation)
def baseline_estimatator(img):
    horz_proj = np.sum(img, 1)
    lb = np.argmax(horz_proj)
    avg_row_density = np.mean(horz_proj)
    lu = np.argmax(horz_proj >= avg_row_density)
    return lb, lu


def leftUp(img, height):
    k1 = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]
    ])
    k2 = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 1, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def rightUp(img, height):
    k1 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0]
    ])
    k2 = np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def rightDown(img, height):
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])
    k2 = np.array([
        [0, 1, 1],
        [1, 1, 0],
        [1, 0, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def leftDown(img, height):
    k1 = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]
    ])
    k2 = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def vertical(img, height):
    k1 = np.array([
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]
    ])
    k2 = np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def horizontal(img, height):
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 1]
    ])
    k2 = np.array([
        [0, 0, 0],
        [1, 1, 1],
        [0, 0, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def get_center_of_mass(window):
    # calculate moments of binary image
    M = cv2.moments(window)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / (M["m00"] + 1e-5))
    cY = int(M["m01"] / (M["m00"] + 1e-5))
    return cX, cY

def getImageFeatures(img, WIDTH, HEIGHT, STRID, N, baseline):
    lb, lu = baseline
    img_height = img.shape[0]
    img_width = img.shape[1]
    X = []
    centers = []
    for r in range(img_width - 1, WIDTH - 1, -STRID):
        window = img[:, r-WIDTH:r]
        X_WINDOW = []
        centers.append(get_center_of_mass(window))
        X_WINDOW.append(np.sum(img[0:lb-1, r-WIDTH:r] == 255) / (img_height * WIDTH))
        X_WINDOW.append(np.sum(img[lb:, r-WIDTH:r] == 255) /
                  ((img_height - lb) * WIDTH))
        X_WINDOW.append(np.sum(window == 255) / (WIDTH * img_height))
        X_WINDOW.append(leftUp(window, img_height))
        X_WINDOW.append(rightUp(window, img_height))
        X_WINDOW.append(rightDown(window, img_height))
        X_WINDOW.append(leftDown(window, img_height))
        X_WINDOW.append(vertical(window, img_height))
        X_WINDOW.append(horizontal(window, img_height))
        window = img[lu:lb, r-WIDTH:r]
        X_WINDOW.append(leftUp(window, np.abs(lb-lu)))
        X_WINDOW.append(rightUp(window, np.abs(lb-lu)))
        X_WINDOW.append(rightDown(window, np.abs(lb-lu)))
        X_WINDOW.append(leftDown(window, np.abs(lb-lu)))
        X_WINDOW.append(vertical(window, np.abs(lb-lu)))
        X_WINDOW.append(horizontal(window, np.abs(lb-lu)))
        window = img[lu:lb, r-WIDTH:r]
        for w in range(WIDTH):
            X_WINDOW.append(np.sum(window[:, w]) / img_height)
        contours, _ = cv2.findContours(
            window.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        X_WINDOW.append(len(contours))
        X_WINDOW.append(np.average(cv2.Canny(window, 50, 200) == 255))
        X.append(X_WINDOW)
    X[0].append(centers[0][0])
    X[0].append(centers[0][1])
    for i in range(1, len(centers)):
        X[i].append(centers[i][0] - centers[i - 1][0])
        X[i].append(centers[i][1] - centers[i - 1][1])
    for i in range(0, len(centers)):
        X[i].append((centers[i][1] - lb) / img_height)
    for inx, vec in enumerate(X):
        X[inx] = np.array(vec)
    return np.array(X, dtype=object)


In [ ]:
from itertools import groupby


def runs_of_ones_list(bits):
    return np.array([sum(g) for b, g in groupby(bits) if b], dtype=object)
print(runs_of_ones_list(X_train[0][40,:]).shape[0])



(3,)


In [97]:
data_set = []
Y = []
for i in range(1, 10):
    for filename in os.listdir("ACdata_base/" + str(i)):
        img = cv2.imread(os.path.join("ACdata_base/" + str(i),
                                      filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            data_set.append(pre_processing(img))
            Y.append(i)
X_train, X_test, Y_train, Y_test = train_test_split(
    data_set, Y, test_size=0.2, random_state=85)


def getFeatures28(images):
    N = 20
    WIDTH = 32 
    STRID = 8
    x_features = []
    for i in range(len(images)):
            img = images[i]
            x_features.append((getImageFeatures(img, WIDTH, int(img.shape[0] / N), STRID, N, baseline_estimatator(img)), Y_train[i]))
    return np.asarray(x_features, dtype=object)



features28_training = getFeatures28(X_train)

print(features28_training.shape)




(1348, 2)


In [98]:
X = [[] for _ in range(9)]
for (mat, inx) in features28_training:
    for vec in mat:
        X[inx - 1].append(vec)

In [99]:
X1 = np.array(X[0])
X2 = np.array(X[1])
X3 = np.array(X[2])
X4 = np.array(X[3])
X5 = np.array(X[4])
X6 = np.array(X[5])
X7 = np.array(X[6])
X8 = np.array(X[7])
X9 = np.array(X[8])

In [96]:
# np.savetxt("sample_42_X1.csv", X[0], delimiter=",")
# np.savetxt("sample_42_X2.csv", X[1], delimiter=",")
# np.savetxt("sample_42_X3.csv", X[2], delimiter=",")
# np.savetxt("sample_42_X4.csv", X[3], delimiter=",")
# np.savetxt("sample_42_X5.csv", X[4], delimiter=",")
# np.savetxt("sample_42_X6.csv", X[5], delimiter=",")
# np.savetxt("sample_42_X7.csv", X[6], delimiter=",")
# np.savetxt("sample_42_X8.csv", X[7], delimiter=",")
np.savetxt("sample_X1.csv", X[0], delimiter=",")
np.savetxt("sample_X2.csv", X[1], delimiter=",")
np.savetxt("sample_X3.csv", X[2], delimiter=",")
np.savetxt("sample_X4.csv", X[3], delimiter=",")
np.savetxt("sample_X5.csv", X[4], delimiter=",")
np.savetxt("sample_X6.csv", X[5], delimiter=",")
np.savetxt("sample_X7.csv", X[6], delimiter=",")
np.savetxt("sample_X8.csv", X[7], delimiter=",")


In [106]:
class Model:
    CLASSES = 9
    def __init__(self):
        self.models = [GaussianMixture(n_components=6, max_iter=200)
                       for _ in range(Model.CLASSES)]
    def fit(self, X, y):
        self.models[y].fit(X)
    def pridect(self, X):
        label = None
        global_likelihood = float('-inf')
        for inx, model in enumerate(self.models):
            local_likelihoods = model.score_samples(X)
            temp_global_likelihood = sum(local_likelihoods)

            if temp_global_likelihood > global_likelihood:
                label = inx
                global_likelihood = temp_global_likelihood
        return label
    def save(self):
        for i in range(Model.CLASSES):
            np.save('gmm_' + str(i) + '_weights', self.models[i].weights_, allow_pickle=False)
            np.save('gmm_' + str(i) + '_means', self.models[i].means_, allow_pickle=False)
            np.save('gmm_' + str(i) + '_covariances', self.models[i].covariances_, allow_pickle=False)
    def reload(self):
        self.models = []
        for i in range(Model.CLASSES):
            means = np.load('gmm_' + str(i) + '_means.npy')
            covar = np.load('gmm_' + str(i) + '_covariances.npy')
            loaded_gmm = GaussianMixture(
                n_components=len(means), covariance_type='full')
            loaded_gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
            loaded_gmm.weights_ = np.load('gmm_' + str(i) + '_weights.npy')
            loaded_gmm.means_ = means
            loaded_gmm.covariances_ = covar
            self.models.append(loaded_gmm)


In [115]:
M = Model()
M.models = None
M.reload()
print(M.models)

None


In [111]:
M = Model()
M.fit(X1, 0)
M.fit(X2, 1)
M.fit(X3, 2)
M.fit(X4, 3)
M.fit(X5, 4)
M.fit(X6, 5)
M.fit(X7, 6)
M.fit(X8, 7)
M.fit(X9, 8)



In [44]:
img = X_test[3]
X_t = getImageFeatures(img, 32, int(
    img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    
print(Y_test[3])
print(M.pridect(X_t) + 1)


1
1


In [95]:

ac = 0
for inx, img in enumerate(X_train):
    X_t = getImageFeatures(img, 32, int(
        img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    if M.pridect(X_t) + 1 == Y_train[inx]:
        ac +=1
print(ac/len(Y_train) * 100)


97.55192878338279


In [112]:
ac = 0
for inx, img in enumerate(X_test):
    X_t = getImageFeatures(img, 32, int(
        img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    if M.pridect(X_t) + 1 == Y_test[inx]:
        ac += 1
print(ac/len(Y_test) * 100)


95.25222551928783
